original dataset: Opinosis Opinion ⁄ Review Data Set https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

In [23]:
# encoding: utf-8

import numpy as np
import pandas as pd
import string
import itertools
import hashlib
import random

In [24]:
# 8 text files we choose to use
file_list = ['battery-life_amazon_kindle',
            'battery-life_ipod_nano_8gb',
            'location_bestwestern_hotel_sfo',
            'location_holiday_inn_london',
            'price_amazon_kindle',
            'room_holiday_inn_london',
            'rooms_bestwestern_hotel_sfo',
            'screen_ipod_nano_8gb']

In [25]:
# I/O function
def import_dataset(file_name):
    """
    input: file_name in file_list
    """
    
    dataFile = './dataset/' + str(file_name) + '.txt.data'
    with open(dataFile, 'rb') as f:
        contents = []
        for line in f.readlines():
            contents.append(line.strip().decode('utf-8'))
    return contents

In [26]:
# import dataset
print('Importing dataset')

dataset = [] # 8-entry list, each corresponding to a file
for file_name in file_list:
    contents = import_dataset(file_name)
    dataset.append(contents)
    
print('document number:' + str(len(dataset)))

Importing dataset
document number:8


1. A class Shingling that constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.

In [27]:
# function constructs k-shingles from a given text
def shingling(text, k=5):
    """
    input: text in dataset
            k, shingle size.
    """
    
    # split the string into separate words
    split_text = []
    for review in text:
        exclude = set(string.punctuation)
        review = ''.join(ch for ch in review if ch not in exclude) # remove punctuation
        review = review.lower() # convert all characters into lower characters
        #for word in review.split():
        #    split_text.append(word)
        for character in list(review):
            split_text.append(character)
    
    # k-shingle
    shingle_list = []
    for i in range(len(split_text)-k+1):
        shingle_list.append(split_text[i:i+k])

    # remove duplicates
    shingle_list.sort()
    shingle_no_dup = list(shingle_list for shingle_list,_ in itertools.groupby(shingle_list))
    
    # each sublist in shingle_no_dup represents a shingle
    # convert the sublist into a string
    shingle_strings = []
    for index, shingle in enumerate(shingle_no_dup):
        sum_string = shingle[0]
        for i in range(1, len(shingle)):
            sum_string = sum_string + ' ' + shingle[i]
        shingle_strings.append(sum_string)
        
    return shingle_strings

In [28]:
# shingling all 8 texts
print('Shingling')

shingle_texts = []
for text in dataset:
    shingle_texts.append(shingling(text))
    
for i in range(len(shingle_texts)):
    num = len(shingle_texts[i])
    print(f'number of shingles in document {i+1} is {num}')

Shingling
number of shingles in document 1 is 4838
number of shingles in document 2 is 3228
number of shingles in document 3 is 8507
number of shingles in document 4 is 9910
number of shingles in document 5 is 5360
number of shingles in document 6 is 17206
number of shingles in document 7 is 9323
number of shingles in document 8 is 3247


In [29]:
# flatting the list of all shingles
flat_shingle_texts = np.hstack(np.array(shingle_texts))
print('number of shingles:',flat_shingle_texts.shape[0])

# get unique shingles
unique_flat_shingle_texts = np.unique(flat_shingle_texts)
max_value = unique_flat_shingle_texts.shape[0]
print('number of unique shingles:', max_value)

number of shingles: 61619
number of unique shingles: 30623


In [30]:
# hash unique shingles
# build dictionary with {unique shingle: hash value}
shingle_dict = {}
for i in range(len(unique_flat_shingle_texts)):
    shingle_dict[unique_flat_shingle_texts[i]] = hash(unique_flat_shingle_texts[i])

In [31]:
# represents the document in the form of an ordered set of its hashed k-shingles
shingle_int = shingle_texts
for i in range(len(shingle_int)):
    for j in range(len(shingle_int[i])):
        shingle_int[i][j] = shingle_dict[shingle_texts[i][j]]

2. A class CompareSets that computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.

In [32]:
# compute Jaccard similarity of two sets of hashed shingles
def jaccard_similarity(text1, text2):
    """
    input: text1, text2 are entries in shingle_int. eg: text1 = shingle_int[0]
    """
    set1 = set(text1)
    set2 = set(text2)
    jaccard = len(set1.intersection(set2)) / len(set1.union(set2))
    return jaccard

In [33]:
# test example
i = 0 # doc1 index
j = 1 # doc2 index
jaccard = jaccard_similarity(shingle_int[i],shingle_int[j])

namei = file_list[i] # doc1 name
namej = file_list[j] # doc2 name
print(f"Jaccard similarity for document '{namei}' and '{namej}' is {jaccard}.")

Jaccard similarity for document 'battery-life_amazon_kindle' and 'battery-life_ipod_nano_8gb' is 0.17323636363636363.


In [34]:
# build Jaccard similarity matrix
jaccard_similarity_matrix = np.zeros((len(file_list), len(file_list)))
for i in range(len(file_list)):
    for j in range(len(file_list)):
        jaccard_similarity_matrix[i][j] = jaccard_similarity(
            shingle_int[i],shingle_int[j])

In [71]:
for i in range(len(jaccard_similarity_matrix)):
    jaccard_similarity_matrix[i] = [ '%.3f' % elem for elem in jaccard_similarity_matrix[i] ]

In [72]:
print(jaccard_similarity_matrix)

[[1.    0.173 0.147 0.156 0.199 0.149 0.151 0.141]
 [0.173 1.    0.123 0.125 0.149 0.108 0.122 0.225]
 [0.147 0.123 1.    0.291 0.166 0.231 0.294 0.114]
 [0.156 0.125 0.291 1.    0.174 0.286 0.266 0.122]
 [0.199 0.149 0.166 0.174 1.    0.163 0.172 0.144]
 [0.149 0.108 0.231 0.286 0.163 1.    0.286 0.1  ]
 [0.151 0.122 0.294 0.266 0.172 0.286 1.    0.115]
 [0.141 0.225 0.114 0.122 0.144 0.1   0.115 1.   ]]


3. A class MinHashing that builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).

In [36]:
# test prime
def isPrime(c):
    """
    input: n, an arbitrary number to test prime.
    """
    if c==2 or c==3: return True
    if c%2==0 or c<2: return False
    for i in range(3, int(c**0.5)+1, 2):   # only odd numbers
        if c%i==0:
            return False    
    return True

In [37]:
# generate n random hash functions in the form ax+b mod c
# where c is a prime number
def random_hash(n, max_value=100000):
    """
    input: argument n, number of hash functions.
            max_value, maximum value can be picked randomly.
    """
    
    rand_list = np.zeros((n,3)) # nX3 matrix, columns corresponding to a, b and c
    rand_list = rand_list.tolist()
    
    for i in range(n):
        a = random.randint(1, max_value) 
        b = random.randint(1, max_value) 
        primes = [i for i in range(1,max_value) if isPrime(i)]
        c = random.choice(primes) 
        
        while [a,b,c] in rand_list:
            a = random.randint(1, max_value) 
            b = random.randint(1, max_value) 
            primes = [i for i in range(1,max_value) if isPrime(i)]
            c = random.choice(primes) 
        
        rand_list[i] = [a, b, c]
    
    return rand_list

In [38]:
# minhashing: build signature matrix
# SIG(i,c) is element for the ith hash function and column c
num_hash = 1000
rand_list = random_hash(num_hash) # each entry is a [a, b, c] list

signature_matrix = np.zeros((num_hash, len(file_list)))
for j, text in enumerate(shingle_int): # for each column/text
    #print(j)
    
    for i in range(num_hash): # for each row/hash function
        minhash = np.inf # initially set SIG(i,c) to inf for all
        for element in text: # iteratively compare and keep the smaller of SIG(i,c) and hi(r)
            new_hash = (rand_list[i][0] * element + rand_list[i][1]) % rand_list[i][2] # ax+b mod c
            #print(rand_list[i], new_hash)
            
            if new_hash < minhash:
                minhash = new_hash
                #print(minhash)
                
        signature_matrix[i][j] = minhash
print(f'shape of signature matrix: {signature_matrix.shape}')

shape of signature matrix: (1000, 8)


4. A class CompareSignatures that estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.

In [39]:
# compare columns in signature matrix
def vector_similarity(vector1, vector2):
    count = 0
    for i in range(len(vector1)):
        if vector1[i] == vector2[i]:
            count += 1
    return count/len(vector1)

In [40]:
# build signature similarity matrix
signature_similarity_matrix = np.zeros((len(file_list), len(file_list)))
trans_signature_matrix = np.transpose(signature_matrix)
for i in range(len(file_list)):
    for j in range(len(file_list)):
        signature_similarity_matrix[i][j] = vector_similarity(
            trans_signature_matrix[i], trans_signature_matrix[j])

In [41]:
print(signature_similarity_matrix)

[[1.    0.293 0.307 0.304 0.316 0.296 0.295 0.26 ]
 [0.293 1.    0.24  0.238 0.251 0.239 0.237 0.33 ]
 [0.307 0.24  1.    0.425 0.299 0.41  0.432 0.236]
 [0.304 0.238 0.425 1.    0.33  0.479 0.414 0.237]
 [0.316 0.251 0.299 0.33  1.    0.311 0.305 0.227]
 [0.296 0.239 0.41  0.479 0.311 1.    0.458 0.232]
 [0.295 0.237 0.432 0.414 0.305 0.458 1.    0.226]
 [0.26  0.33  0.236 0.237 0.227 0.232 0.226 1.   ]]


5. (Optional task for extra 2 bonus) A class LSH that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds all candidate pairs of signatures that agree on at least fraction t of their components.

In [75]:
def lsh(bands, signature_matrix=signature_matrix, shingle_int=shingle_int):
    """
    input argument: bands.
    """
    rows = num_hash/bands # number of rows in each band
    t = np.power(1/bands, 1/rows) # threshold
    rand_list_lsh = random_hash(bands) # each band has a unique hash function
    candidate_pair_index = []
    
    for i in range(bands): # loop over all bands
        bucket = [] # bucket for the ith band
        a = rand_list_lsh[i][0] # hash function ax+b mod c, x is a vector
        b = rand_list_lsh[i][1]
        c = rand_list_lsh[i][2]
        
        for j in range(len(file_list)): # loop over all columns
            value = 0
            for r in range(i, int(i+rows)): # loop over all rows in a band
                value += a * signature_matrix[r][j] + b
            hash_value = value % c
            bucket.append(hash_value) # a hashed value corresponding to vector of j-th column and i-th band
            
        # compare every two entries in bucket to find candidate pairs
        for m in range(len(bucket)):
            for n in range(m+1, len(bucket)):
                if bucket[m] == bucket[n]:
                    # check if they are candidate pair and we already computed similarity
                    if [m,n] not in candidate_pair_index and [n,m] not in candidate_pair_index:
                        vector_m = signature_matrix[:,m] 
                        vector_n = signature_matrix[:,n]
                        sim = vector_similarity(vector_m, vector_n) # compute signature similarity
                        if sim > t: # above threshold indicates two columns are similar
                            print(f"""Document D{m+1} and D{n+1} are similar. Estimation of similarity through minhash is {sim}. Jaccard similarity is {jaccard_similarity(shingle_int[m], shingle_int[n])}.""")
                        candidate_pair_index.append([m,n])
    return

In [76]:
lsh(bands=200)

Document D6 and D7 are similar. Estimation of similarity through minhash is 0.458. Jaccard similarity is 0.2858181465684374.
Document D3 and D7 are similar. Estimation of similarity through minhash is 0.432. Jaccard similarity is 0.2938103185545316.
Document D4 and D6 are similar. Estimation of similarity through minhash is 0.479. Jaccard similarity is 0.28572783309625416.
Document D4 and D7 are similar. Estimation of similarity through minhash is 0.414. Jaccard similarity is 0.26566201632008424.
Document D3 and D6 are similar. Estimation of similarity through minhash is 0.41. Jaccard similarity is 0.23146551724137931.
Document D3 and D4 are similar. Estimation of similarity through minhash is 0.425. Jaccard similarity is 0.2905192348118562.
